In [97]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import gmaps
# Import API key
from api_keys import gkey
import json

# Incorporated weather Data into dataframe
CityDataFrame = pd.read_csv('weather_dataframe.csv') 

In [98]:
#Visual
CityDataFrame.reset_index()  
CityDataFrame = CityDataFrame.drop(columns={'Unnamed: 0'}) 
CityDataFrame.head()

,city name,Lat,Lng,MaxTemp,Humidity,Cloudiness,WindSpeed,Date,country
0,ixtapa,20.7000,-105.2000,78.80,53,1,4.61,1612032599,MX
1,hasaki,35.7333,140.8333,28.40,68,20,3.44,1612032599,JP
2,jiazi,22.8779,116.0670,55.51,88,3,11.56,1612032599,CN
3,atuona,-9.8000,-139.0333,78.78,78,19,16.11,1612032600,PF
4,berdigestyakh,62.0989,126.6992,-50.01,76,67,1.19,1612032600,RU


In [99]:
#Gmap to Gkey
gmaps.configure(api_key=gkey)

In [100]:
#Base Humidity Map
locations = CityDataFrame[["Lat", "Lng"]].astype(float) 
Humidity = CityDataFrame['Humidity'].astype(float) 
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#Setting conditions for ideal weather
IdealWeather = CityDataFrame.loc[CityDataFrame["MaxTemp"] < 80 ] 
IdealWeather =  CityDataFrame.loc[CityDataFrame["MaxTemp"] > 70 ]  
IdealWeather = CityDataFrame.loc[CityDataFrame["WindSpeed"] < 10]   
IdealWeather = CityDataFrame.loc[CityDataFrame["Cloudiness"]  == 0]    
IdealWeather = IdealWeather.drop(columns = {'Unnamed: 0'}) 
IdealWeather.dtypes 
IdealWeather = IdealWeather.dropna() 
IdealWeather

In [102]:
#New Locations and Humidity
locations = IdealWeather[["Lat", "Lng"]].astype(float)  
Humidity = IdealWeather['Humidity'].astype(float) 

In [103]:
#Specfied Heat Map with ideal weather
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [108]:
# Finding hotels for each location with iterrows
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": gkey
}

# Use the lat/lng to find hotel nearby
for index, row in IdealWeather.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Params and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        IdealWeather.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("oops")

C:\Users\Gowtham\anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Gowtham\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops


In [ ]:
#Infobox code
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""

In [ ]:
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in IdealWeather.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [113]:
#Hotel map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[info_box_template.format(**row) for index, row in IdealWeather.iterrows()]
) 
fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))